In [ ]:
# ---------------------------------
# Parameter Tuning.ipynb
# ---------------------------------

# 1) Imports (same as in main.ipynb)
import pandas as pd
import numpy as np
import os
from scipy.optimize import minimize
import itertools
import matplotlib.pyplot as plt

# Utility Function Imports
from utilities import *

# Algorithm Imports
from Strategies.benchmarks import *
from Strategies.follow_the_loser import *
from Strategies.follow_the_winner import *
from Strategies.pattern_matching import *
from Strategies.meta_learning import *

pd.set_option('future.no_silent_downcasting', True)

# ---------------------------------
# PART 1: Parameter Tuning Setup
# ---------------------------------

# Initialize a uniform portfolio
price_relative_df = pd.read_csv("price_relative_vectors.csv", index_col = 0)
b = initialize_portfolio(price_relative_df.shape[1])

# Build price relative vectors
price_relative_vectors_array = price_relative_df.values
dates = price_relative_df.index

# ---------------------------------
# The Evaluate/Tune Functions
# ---------------------------------

def evaluate_fr_leader(b, price_relative_vectors, beta_val, delta_val, freq=252, risk_free_rate=0.0):
    """
    1) Runs the Follow-the-Regularized-Leader strategy with given beta, delta.
    2) Returns relevant performance metrics.
    """
    # Run strategy
    b_n = follow_the_regularized_leader(b, price_relative_vectors, 
                                        beta=beta_val, 
                                        delta=delta_val)
    
    # Final wealth
    final_wealth = calculate_cumulative_wealth(b_n, price_relative_vectors)

    # Exponential growth
    n_periods = len(price_relative_vectors)
    exp_growth = calculate_exponential_growth_rate(final_wealth, n_periods)

    # Sharpe ratio
    cum_wealth = calculate_cumulative_wealth_over_time(b_n, price_relative_vectors)
    daily_returns = compute_periodic_returns(cum_wealth)
    sharpe = compute_sharpe_ratio(daily_returns, freq=freq, risk_free_rate=risk_free_rate)

    return {
        'beta': beta_val,
        'delta': delta_val,
        'final_wealth': final_wealth,
        'exp_growth': exp_growth,
        'sharpe': sharpe
    }

def tune_follow_the_regularized_leader(b, price_relative_vectors):
    # Example grids for beta and delta
    beta_values = [0.01, 0.05, 0.1, 0.5]
    delta_values = [0.1, 0.5, 1.0]

    results = []
    for (beta_val, delta_val) in itertools.product(beta_values, delta_values):
        perf = evaluate_fr_leader(
            b, 
            price_relative_vectors, 
            beta_val=beta_val, 
            delta_val=delta_val
        )
        results.append(perf)

    # Sort by Sharpe ratio descending
    results.sort(key=lambda x: x['sharpe'] if x['sharpe'] is not None else -9999, reverse=True)
    return results

# ---------------------------------
# Run the tuning
# ---------------------------------
tuning_results = tune_follow_the_regularized_leader(b, price_relative_vectors_array)

# Display results
print("=== Tuning Results ===")
for row in tuning_results:
    print(
        f"beta={row['beta']}, delta={row['delta']}, "
        f"final_wealth={row['final_wealth']:.4f}, "
        f"exp_growth={row['exp_growth']:.4f}, "
        f"sharpe={row['sharpe']:.4f}"
    )


                AAPL      ADBE       ADI       ADP      ADSK       AEP  \
Date                                                                     
2020-03-20  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
2020-03-23  1.001272  1.064626  1.013781  0.990994  0.990994  0.998042   
2020-03-24  1.077576  0.989776  1.111111  1.131817  1.038761  1.010376   
2020-03-25  1.006179  0.987088  0.996277  1.002895  0.993306  1.017210   
2020-03-26  1.045977  1.043492  0.989002  1.115085  1.076266  1.114477   
...              ...       ...       ...       ...       ...       ...   
2021-03-12  0.990833  0.985161  0.976153  1.011735  0.987677  1.014259   
2021-03-15  1.025277  1.008318  1.013423  1.034524  1.025402  1.013688   
2021-03-16  1.008540  1.010033  1.013775  0.995293  1.000000  1.014599   
2021-03-17  0.998003  0.997792  1.007055  0.998353  0.994863  0.988849   
2021-03-18  0.966141  0.974558  0.971393  0.990313  0.980116  0.998909   

                ALGN      AMAT       

In [2]:
price_relative_df

,AAPL,ADBE,ADI,ADP,ADSK,AEP,ALGN,AMAT,AMD,AMGN,...,SBUX,SNPS,SWKS,TMUS,TSLA,TXN,VRSK,VRSN,VRTX,XEL
Date,,,,,,,,,,,,,,,,,,,,,
2020-03-20,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2020-03-23,1.001272,1.064626,1.013781,0.990994,0.990994,0.998042,0.955933,1.087877,1.070791,1.018319,...,1.004656,0.986182,1.043247,1.030319,1.050970,1.031093,0.975292,0.962388,1.006098,1.000000
2020-03-24,1.077576,0.989776,1.111111,1.131817,1.038761,1.010376,1.200000,1.093676,1.087204,1.070586,...,1.107964,1.112002,1.141362,1.017318,1.143967,1.064692,1.151760,1.127695,1.081842,1.080800
2020-03-25,1.006179,0.987088,0.996277,1.002895,0.993306,1.017210,1.143315,1.010011,0.993243,0.954825,...,1.025562,1.004974,0.996563,1.016255,1.061825,0.933817,0.961019,0.979124,0.942276,1.027941
2020-03-26,1.045977,1.043492,0.989002,1.115085,1.076266,1.114477,1.010635,1.089866,1.043450,1.033491,...,1.061179,1.062938,1.075995,1.042569,0.974122,1.067068,1.071426,1.053483,1.070155,1.042485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-12,0.990833,0.985161,0.976153,1.011735,0.987677,1.014259,0.983456,0.968729,0.989877,0.996286,...,1.001766,0.968395,0.992196,1.000314,0.992550,0.993265,0.988439,0.984346,0.995818,1.019566
2021-03-15,1.025277,1.008318,1.013423,1.034524,1.025402,1.013688,1.047888,1.009710,1.024150,1.024585,...,1.008628,1.026057,1.034124,0.986254,1.013265,1.005057,1.020994,1.013244,1.019599,1.008323
2021-03-16,1.008540,1.010033,1.013775,0.995293,1.000000,1.014599,0.970354,1.031017,0.999880,1.025785,...,1.025294,0.996900,1.017338,1.022220,0.960128,1.008404,0.990721,1.003307,1.002288,1.007937


: 